In [1]:
# Run this file first to scrape Pokemon tier data from smogon.com, combine the data with pokemon and battle stats from Kaggle, and generate a sqlite db

In [2]:
# Import relevant libraries
import requests
import re
import json
import pandas as pd
import numpy as np

In [3]:
# Web scraping by Lea Jinks

In [4]:
# this cell modified from https://stackoverflow.com/q/60017438/19966544
response = requests.get('https://www.smogon.com/dex/xy/pokemon/')

# pull JSON from response text via regex
data = "".join(re.findall(r'dexSettings = (\{.*\})', response.text))

# parse JSON
data = json.loads(data)

# query JSON to return relevant info
data = data.get('injectRpcs', [])[1][1].get('pokemon')

In [5]:
# make list of pokemon names
name_list = []
for row in data:
  name = row.get('name')

  # make names match original csv by removing hyphens and 
  x = name.split('-')
  if "Mega" in x:
    x.reverse()
    result = " ".join(x)
  else:
    result = " ".join(x)
  name_list.append(result)

# make list of tiers
tier_list = []
for row in data:
  tier_list.append(row.get('formats'))

In [6]:
# create df with name and tier values
tier_df = pd.DataFrame(name_list, columns = ["Name"])
tier_df.insert(1, "Tier", tier_list)
tier_df['Tier'] = tier_df['Tier'].str[0]
tier_df

,Name,Tier
0,Bulbasaur,LC
1,Ivysaur,NFE
2,Venusaur,RU
3,Charmander,LC
4,Charmeleon,NFE
...,...,...
876,Darmanitan Zen,UU
877,Mega Kangaskhan,Uber
878,Meloetta Pirouette,RU
879,Mega Aerodactyl,UU


In [7]:
# import kaggle pokemon info dataset
pokemon_df = pd.read_csv('Resources/pokemon.csv')
pokemon_df

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


In [8]:
# standardize pokenames and combine df's 
pokemon_df['Name'] = pokemon_df['Name'].str.replace('Forme', '')
pokemon_df['Name'] = pokemon_df['Name'].str.replace('Size', '')
pokemon_df['Name'] = pokemon_df['Name'].str.replace('Mode', '')

In [9]:
# merge kaggle data with scraped tiers data
pokemon_df = pokemon_df.merge(tier_df, on='Name')

In [10]:
# export to json for JS formatting
pokemon_df.to_json('Resources/pokemon_with_tiers.json', orient="records", indent=4)

In [11]:
pokemon_df = pokemon_df.reset_index()

In [12]:
# rename columns to be compatible with sqlite
pokemon_df = pokemon_df.rename(columns={'#': 'Number', 'index':'_ID', 'Type 1':'Type_1', 'Type 2':'Type_2', "Sp. Atk": "Sp_Atk", 'Sp. Def': 'Sp_Def' })
pokemon_df

,_ID,Number,Name,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary,Tier
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,LC
1,1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,NFE
2,2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,RU
3,3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,OU
4,4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,LC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,739,794,Yveltal,Dark,Flying,126,131,95,131,98,99,6,True,Uber
740,740,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True,RU
741,741,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True,OU
742,742,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True,Uber


In [13]:
# create sqlite file for flask app by Becky Klosowski

In [14]:
import sqlite3
from pathlib import Path

In [15]:
database_path = "Resources/pokemon_with_tiers.sqlite"
Path(database_path).touch()

In [16]:
conn = sqlite3.connect(database_path)
c = conn.cursor()

In [17]:
c.execute('''DROP TABLE IF EXISTS poketiers;''')

In [18]:
c.execute('''CREATE TABLE poketiers(
    _ID INT NOT NULL PRIMARY KEY,
    Number INT NOT NULL,
  	Name VARCHAR(40) NOT NULL,
    Type_1 VARCHAR(40),
	Type_2 VARCHAR(40),
	HP INT NOT NULL,
	Attack INT NOT NULL,
	Defense INT NOT NULL,
	Sp_Atk INT NOT NULL,
	Sp_Def INT NOT NULL,
	Speed INT NOT NULL,
	Generation INT NOT NULL,
	Legendary INT NOT NULL,
	Tier VARCHAR(40) NOT NULL
	);''')

In [19]:
pokemon_df.to_sql("poketiers", conn, if_exists='append', index = False)

In [20]:
c.execute('''SELECT * from poketiers''').fetchone()

(0, 1, 'Bulbasaur', 'Grass', 'Poison', 45, 49, 49, 65, 65, 45, 1, 0, 'LC')

In [21]:
conn.close()

In [22]:
#Data Exploration by Crystal Butler

In [23]:
pokemon_df.head()

,_ID,Number,Name,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary,Tier
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,LC
1,1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,NFE
2,2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,RU
3,3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,OU
4,4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,LC


In [24]:
battles_df = pd.read_csv("Resources/combats.csv")

In [25]:
battles_df = battles_df.sort_values('First_pokemon')

In [26]:
battles_df.head()

,First_pokemon,Second_pokemon,Winner
15176,1,372,372
45814,1,462,462
17362,1,6,6
15251,1,649,649
4883,1,285,1


In [27]:
initial_value = 1
battles_df['Battle_number'] = range(initial_value, len(battles_df) +initial_value)

In [28]:
battles_df.head()

,First_pokemon,Second_pokemon,Winner,Battle_number
15176,1,372,372,1
45814,1,462,462,2
17362,1,6,6,3
15251,1,649,649,4
4883,1,285,1,5


In [29]:
battles_df['Did_the_first_pokemon_win?'] = (battles_df['First_pokemon'] == battles_df['Winner']).astype(int)

In [30]:
battles_df.head()

,First_pokemon,Second_pokemon,Winner,Battle_number,Did_the_first_pokemon_win?
15176,1,372,372,1,0
45814,1,462,462,2,0
17362,1,6,6,3,0
15251,1,649,649,4,0
4883,1,285,1,5,1


In [31]:
battles2_df = battles_df.copy()

In [32]:
battles2_df = battles2_df.iloc[:,[1,0,2,3]]


In [33]:
battles2_df.head()

,Second_pokemon,First_pokemon,Winner,Battle_number
15176,372,1,372,1
45814,462,1,462,2
17362,6,1,6,3
15251,649,1,649,4
4883,285,1,1,5


In [34]:
battles2_df['Did_the_first_pokemon_win?'] = (battles2_df['Second_pokemon'] == battles2_df['Winner']).astype(int)

In [35]:
battles2_df.head()


,Second_pokemon,First_pokemon,Winner,Battle_number,Did_the_first_pokemon_win?
15176,372,1,372,1,1
45814,462,1,462,2,1
17362,6,1,6,3,1
15251,649,1,649,4,1
4883,285,1,1,5,0


In [36]:
battles2_df.rename(columns = {'Second_pokemon':'First_pokemon', 'First_pokemon':'Second_pokemon'}, inplace = True)

In [37]:
battles2_df.head()

,First_pokemon,Second_pokemon,Winner,Battle_number,Did_the_first_pokemon_win?
15176,372,1,372,1,1
45814,462,1,462,2,1
17362,6,1,6,3,1
15251,649,1,649,4,1
4883,285,1,1,5,0


In [38]:
all_battles_df = pd.concat([battles_df, battles2_df], axis=0)

In [39]:
all_battles_df = all_battles_df.sort_values('Battle_number')

In [40]:
all_battles_df.dtypes

First_pokemon                 int64
Second_pokemon                int64
Winner                        int64
Battle_number                 int64
Did_the_first_pokemon_win?    int64
dtype: object

In [41]:
merged_df = pd.merge(all_battles_df,pokemon_df, how='inner', left_on=['First_pokemon'], right_on = ['Number'])
merged_df.head()

,First_pokemon,Second_pokemon,Winner,Battle_number,Did_the_first_pokemon_win?,_ID,Number,Name,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary,Tier
0,1,372,372,1,0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,LC
1,1,462,462,2,0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,LC
2,1,6,6,3,0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,LC
3,1,649,649,4,0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,LC
4,1,285,1,5,1,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,LC


In [42]:
merged_df = merged_df.iloc[:,[3,0,7,1,2,4]]

In [43]:
merged_df.rename(columns = {'Name':'First_name'},inplace = True)

In [44]:
merged_df.head()

,Battle_number,First_pokemon,First_name,Second_pokemon,Winner,Did_the_first_pokemon_win?
0,1,1,Bulbasaur,372,372,0
1,2,1,Bulbasaur,462,462,0
2,3,1,Bulbasaur,6,6,0
3,4,1,Bulbasaur,649,649,0
4,5,1,Bulbasaur,285,1,1


In [45]:
merged_df = pd.merge(merged_df,pokemon_df, how='inner', left_on=['Second_pokemon'], right_on = ['Number'])

In [46]:
merged_df.head()

,Battle_number,First_pokemon,First_name,Second_pokemon,Winner,Did_the_first_pokemon_win?,_ID,Number,Name,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary,Tier
0,1,1,Bulbasaur,372,372,0,363,372,Barboach,Water,Ground,50,48,43,46,41,60,3,False,LC
1,23232,638,Gothitelle,372,638,1,363,372,Barboach,Water,Ground,50,48,43,46,41,60,3,False,LC
2,24480,391,Tropius,372,372,0,363,372,Barboach,Water,Ground,50,48,43,46,41,60,3,False,LC
3,23218,558,Tepig,372,372,0,363,372,Barboach,Water,Ground,50,48,43,46,41,60,3,False,LC
4,19353,311,Breloom,372,311,1,363,372,Barboach,Water,Ground,50,48,43,46,41,60,3,False,LC


In [47]:
merged_df.rename(columns = {'Name':'Second_name'},inplace = True)

In [48]:
merged_df = merged_df.iloc[:,[0,1,2,3,8,4,5]]

In [49]:
merged_df = merged_df.sort_values('Battle_number')

In [50]:
merged_df.head()

,Battle_number,First_pokemon,First_name,Second_pokemon,Second_name,Winner,Did_the_first_pokemon_win?
0,1,1,Bulbasaur,372,Barboach,372,0
14008,1,372,Barboach,1,Bulbasaur,372,1
14009,2,462,Mothim,1,Bulbasaur,462,1
140,2,1,Bulbasaur,462,Mothim,462,0
14010,3,6,Charmeleon,1,Bulbasaur,6,1


In [51]:
winner_df_names = merged_df.iloc[:,[0,2,4,6]]

In [52]:
winner_df_names.head()

,Battle_number,First_name,Second_name,Did_the_first_pokemon_win?
0,1,Bulbasaur,Barboach,0
14008,1,Barboach,Bulbasaur,1
14009,2,Mothim,Bulbasaur,1
140,2,Bulbasaur,Mothim,0
14010,3,Charmeleon,Bulbasaur,1


In [53]:
#merging pokemon_df and merged_df dataframe, so First Name can have it's attributes associated with the pokemon that have battled
merged_w_tiers = pd.merge(pokemon_df,merged_df, how='left', left_on=['Name'], right_on=['First_name'])

In [54]:
#viewing dataframe
merged_w_tiers.head()

,_ID,Number,Name,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,...,Generation,Legendary,Tier,Battle_number,First_pokemon,First_name,Second_pokemon,Second_name,Winner,Did_the_first_pokemon_win?
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,1,False,LC,1.0,1.0,Bulbasaur,372.0,Barboach,372.0,0.0
1,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,1,False,LC,2.0,1.0,Bulbasaur,462.0,Mothim,462.0,0.0
2,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,1,False,LC,3.0,1.0,Bulbasaur,6.0,Charmeleon,6.0,0.0
3,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,1,False,LC,4.0,1.0,Bulbasaur,649.0,Emolga,649.0,0.0
4,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,1,False,LC,5.0,1.0,Bulbasaur,285.0,Poochyena,1.0,1.0


In [55]:
#merging pokemon_df and merged_w_tiers dataframe, so Second Name can have it's attributes as well
merged_w_tiers2 = pd.merge(pokemon_df,merged_w_tiers, how='left', left_on=['Name'], right_on=['Second_name']).sort_values('Battle_number')

In [56]:
#viewing columns for renaming
merged_w_tiers2.columns

Index(['_ID_x', 'Number_x', 'Name_x', 'Type_1_x', 'Type_2_x', 'HP_x',
       'Attack_x', 'Defense_x', 'Sp_Atk_x', 'Sp_Def_x', 'Speed_x',
       'Generation_x', 'Legendary_x', 'Tier_x', '_ID_y', 'Number_y', 'Name_y',
       'Type_1_y', 'Type_2_y', 'HP_y', 'Attack_y', 'Defense_y', 'Sp_Atk_y',
       'Sp_Def_y', 'Speed_y', 'Generation_y', 'Legendary_y', 'Tier_y',
       'Battle_number', 'First_pokemon', 'First_name', 'Second_pokemon',
       'Second_name', 'Winner', 'Did_the_first_pokemon_win?'],
      dtype='object')

In [57]:
#Renaming columns to reflect which attributes go to which Pokemon (First or Second)
merged_w_tiers2 = merged_w_tiers2.rename(columns={'Number_x':'Number_Second', 'Name_x':'Second_Name', 'Type_1_x':'Type_1_Second', 'Type_2_x':'Type_2_Second', 'HP_x':'HP_Second', 'Attack_x':'Attack_Second',
       'Defense_x':'Defense_Second', 'Sp_Atk_x':'Sp_Atk_Second', 'Sp_Def_x':'Sp_Def_Second', 'Speed_x':'Speed_Second', 'Generation_x':'Generation_Second',
       'Legendary_x':'Legendary_Second', 'Tier_x':'Tier_Second', 'Number_y':'Number_First', 'Name_y':'First_Name', 'Type_1_y':'Type_1_First', 'Type_2_y':'Type_2_First',
       'HP_y':'HP_First', 'Attack_y':'Attack_First', 'Defense_y':'Defense_First', 'Sp_Atk_y':'Sp_Atk_First', 'Sp_Def_y':'Sp_Def_First', 'Speed_y':'Speed_First',
       'Generation_y':'Generation_First', 'Legendary_y':'Legendary_First', 'Tier_y':'Tier_First', 'Did_the_first_pokemon_win?':'Did_the_first_pokemon_win' })
merged_w_tiers2

,_ID_x,Number_Second,Second_Name,Type_1_Second,Type_2_Second,HP_Second,Attack_Second,Defense_Second,Sp_Atk_Second,Sp_Def_Second,...,Generation_First,Legendary_First,Tier_First,Battle_number,First_pokemon,First_name,Second_pokemon,Second_name,Winner,Did_the_first_pokemon_win
65,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,3.0,False,LC,1.0,372.0,Barboach,1.0,Bulbasaur,372.0,1.0
42104,363,372,Barboach,Water,Ground,50,48,43,46,41,...,1.0,False,LC,1.0,1.0,Bulbasaur,372.0,Barboach,372.0,0.0
79,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,4.0,False,Untiered,2.0,462.0,Mothim,1.0,Bulbasaur,462.0,1.0
51724,444,462,Mothim,Bug,Flying,70,94,50,94,50,...,1.0,False,LC,2.0,1.0,Bulbasaur,462.0,Mothim,462.0,0.0
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,1.0,False,NFE,3.0,6.0,Charmeleon,1.0,Bulbasaur,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47791,410,419,Mega Latias,Dragon,Psychic,80,100,120,140,150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53730,461,479,Honchkrow,Dark,Flying,100,125,52,105,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61445,528,556,Servine,Grass,NaN,60,60,75,60,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68469,588,618,Maractus,Grass,NaN,75,86,67,106,67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
#Sorting by Battle Number
merged_w_tiers2 = merged_w_tiers2.sort_values('Battle_number')

In [59]:
#Filtering to just Pokemon without battles
orphan_pokemon = merged_w_tiers2[merged_w_tiers2['Battle_number'].isnull()]

In [60]:
#Dataframe of Pokemon that haven't battled

orphan_pokemon.head()

,_ID_x,Number_Second,Second_Name,Type_1_Second,Type_2_Second,HP_Second,Attack_Second,Defense_Second,Sp_Atk_Second,Sp_Def_Second,...,Generation_First,Legendary_First,Tier_First,Battle_number,First_pokemon,First_name,Second_pokemon,Second_name,Winner,Did_the_first_pokemon_win
1057,9,12,Blastoise,Water,NaN,79,83,100,85,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3442,30,33,Sandshrew,Ground,NaN,50,75,85,20,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4635,41,46,Wigglytuff,Normal,Fairy,140,70,45,85,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6830,60,66,Poliwag,Water,NaN,40,50,40,40,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8109,72,78,Victreebel,Grass,Poison,80,105,65,100,70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
#filtering out Pokemon that haven't battled
merged_reduced =merged_w_tiers2.dropna(subset=['Battle_number']).sort_values('Battle_number')

In [62]:
#Dataframe with only Pokemon that were involved in a battle

merged_reduced.head()

,_ID_x,Number_Second,Second_Name,Type_1_Second,Type_2_Second,HP_Second,Attack_Second,Defense_Second,Sp_Atk_Second,Sp_Def_Second,...,Generation_First,Legendary_First,Tier_First,Battle_number,First_pokemon,First_name,Second_pokemon,Second_name,Winner,Did_the_first_pokemon_win
65,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,3.0,False,LC,1.0,372.0,Barboach,1.0,Bulbasaur,372.0,1.0
42104,363,372,Barboach,Water,Ground,50,48,43,46,41,...,1.0,False,LC,1.0,1.0,Bulbasaur,372.0,Barboach,372.0,0.0
79,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,4.0,False,Untiered,2.0,462.0,Mothim,1.0,Bulbasaur,462.0,1.0
51724,444,462,Mothim,Bug,Flying,70,94,50,94,50,...,1.0,False,LC,2.0,1.0,Bulbasaur,462.0,Mothim,462.0,0.0
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,1.0,False,NFE,3.0,6.0,Charmeleon,1.0,Bulbasaur,6.0,1.0


In [63]:
merged_reduced.columns


Index(['_ID_x', 'Number_Second', 'Second_Name', 'Type_1_Second',
       'Type_2_Second', 'HP_Second', 'Attack_Second', 'Defense_Second',
       'Sp_Atk_Second', 'Sp_Def_Second', 'Speed_Second', 'Generation_Second',
       'Legendary_Second', 'Tier_Second', '_ID_y', 'Number_First',
       'First_Name', 'Type_1_First', 'Type_2_First', 'HP_First',
       'Attack_First', 'Defense_First', 'Sp_Atk_First', 'Sp_Def_First',
       'Speed_First', 'Generation_First', 'Legendary_First', 'Tier_First',
       'Battle_number', 'First_pokemon', 'First_name', 'Second_pokemon',
       'Second_name', 'Winner', 'Did_the_first_pokemon_win'],
      dtype='object')

In [64]:
merged_reduced[['Number_Second',
       'HP_Second', 'Attack_Second', 'Defense_Second', 'Sp_Atk_Second',
       'Sp_Def_Second', 'Speed_Second', 'Generation_Second', 'Number_First', 'HP_First', 'Attack_First',
       'Defense_First', 'Sp_Atk_First', 'Sp_Def_First', 'Speed_First',
       'Generation_First', 'Battle_number',
       'First_pokemon', 'Second_pokemon',
       'Winner', 'Did_the_first_pokemon_win']] = merged_reduced[['Number_Second',
       'HP_Second', 'Attack_Second', 'Defense_Second', 'Sp_Atk_Second',
       'Sp_Def_Second', 'Speed_Second', 'Generation_Second', 'Number_First', 'HP_First', 'Attack_First',
       'Defense_First', 'Sp_Atk_First', 'Sp_Def_First', 'Speed_First',
       'Generation_First', 'Battle_number',
       'First_pokemon', 'Second_pokemon',
       'Winner', 'Did_the_first_pokemon_win']].astype(int)

In [65]:
merged_reduced.columns

Index(['_ID_x', 'Number_Second', 'Second_Name', 'Type_1_Second',
       'Type_2_Second', 'HP_Second', 'Attack_Second', 'Defense_Second',
       'Sp_Atk_Second', 'Sp_Def_Second', 'Speed_Second', 'Generation_Second',
       'Legendary_Second', 'Tier_Second', '_ID_y', 'Number_First',
       'First_Name', 'Type_1_First', 'Type_2_First', 'HP_First',
       'Attack_First', 'Defense_First', 'Sp_Atk_First', 'Sp_Def_First',
       'Speed_First', 'Generation_First', 'Legendary_First', 'Tier_First',
       'Battle_number', 'First_pokemon', 'First_name', 'Second_pokemon',
       'Second_name', 'Winner', 'Did_the_first_pokemon_win'],
      dtype='object')

In [66]:
final_df = merged_reduced[['Number_First', 'First_Name',
       'Type_1_First', 'Type_2_First', 'HP_First', 'Attack_First',
       'Defense_First', 'Sp_Atk_First', 'Sp_Def_First', 'Speed_First',
       'Generation_First', 'Legendary_First', 'Tier_First','Number_Second', 'Second_Name', 'Type_1_Second', 'Type_2_Second',
       'HP_Second', 'Attack_Second', 'Defense_Second', 'Sp_Atk_Second',
       'Sp_Def_Second', 'Speed_Second', 'Generation_Second',
       'Legendary_Second', 'Tier_Second',  'Battle_number',
       'Did_the_first_pokemon_win']]

In [67]:
final_df.head()

,Number_First,First_Name,Type_1_First,Type_2_First,HP_First,Attack_First,Defense_First,Sp_Atk_First,Sp_Def_First,Speed_First,...,Attack_Second,Defense_Second,Sp_Atk_Second,Sp_Def_Second,Speed_Second,Generation_Second,Legendary_Second,Tier_Second,Battle_number,Did_the_first_pokemon_win
65,372,Barboach,Water,Ground,50,48,43,46,41,60,...,49,49,65,65,45,1,False,LC,1,1
42104,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,...,48,43,46,41,60,3,False,LC,1,0
79,462,Mothim,Bug,Flying,70,94,50,94,50,66,...,49,49,65,65,45,1,False,LC,2,1
51724,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,...,94,50,94,50,66,4,False,Untiered,2,0
0,6,Charmeleon,Fire,NaN,58,64,58,80,65,80,...,49,49,65,65,45,1,False,LC,3,1


In [68]:
final_df.dtypes


Number_First                  int64
First_Name                   object
Type_1_First                 object
Type_2_First                 object
HP_First                      int64
Attack_First                  int64
Defense_First                 int64
Sp_Atk_First                  int64
Sp_Def_First                  int64
Speed_First                   int64
Generation_First              int64
Legendary_First              object
Tier_First                   object
Number_Second                 int64
Second_Name                  object
Type_1_Second                object
Type_2_Second                object
HP_Second                     int64
Attack_Second                 int64
Defense_Second                int64
Sp_Atk_Second                 int64
Sp_Def_Second                 int64
Speed_Second                  int64
Generation_Second             int64
Legendary_Second               bool
Tier_Second                  object
Battle_number                 int64
Did_the_first_pokemon_win   

In [69]:
final_df.index.name = "_ID"

In [70]:
final_df.columns

Index(['Number_First', 'First_Name', 'Type_1_First', 'Type_2_First',
       'HP_First', 'Attack_First', 'Defense_First', 'Sp_Atk_First',
       'Sp_Def_First', 'Speed_First', 'Generation_First', 'Legendary_First',
       'Tier_First', 'Number_Second', 'Second_Name', 'Type_1_Second',
       'Type_2_Second', 'HP_Second', 'Attack_Second', 'Defense_Second',
       'Sp_Atk_Second', 'Sp_Def_Second', 'Speed_Second', 'Generation_Second',
       'Legendary_Second', 'Tier_Second', 'Battle_number',
       'Did_the_first_pokemon_win'],
      dtype='object')

In [71]:
final_df.to_csv("Resources/final_df.csv")

In [72]:
# sqlite setup by Lea Jinks

In [73]:
database_path = "Resources/pokemon.sqlite"
Path(database_path).touch()

In [74]:
conn = sqlite3.connect(database_path)
c = conn.cursor()

In [75]:
c.execute('''DROP TABLE IF EXISTS pokemon;''')

In [76]:
c.execute('''CREATE TABLE pokemon(
    _ID INT NOT NULL PRIMARY KEY,
    Number_First INT NOT NULL,
  	First_Name VARCHAR(40) NOT NULL,
    Type_1_First VARCHAR(40),
	Type_2_First VARCHAR(40),
	HP_First INT NOT NULL,
	Attack_First INT NOT NULL,
	Defense_First INT NOT NULL,
	Sp_Atk_First INT NOT NULL,
	Sp_Def_First INT NOT NULL,
	Speed_First INT NOT NULL,
	Generation_First INT NOT NULL,
	Legendary_First VARCHAR(40),
	Tier_First VARCHAR(40) NOT NULL,
	Number_Second INT NOT NULL,
	Second_Name VARCHAR(40) NOT NULL,
	Type_1_Second VARCHAR(40),
	Type_2_Second VARCHAR(40),
	HP_Second INT NOT NULL,
	Attack_Second INT NOT NULL,
	Defense_Second INT NOT NULL,
	Sp_Atk_Second INT NOT NULL,
	Sp_Def_Second INT NOT NULL,
	Speed_Second INT NOT NULL,
	Generation_Second INT NOT NULL,
	Legendary_Second VARCHAR(40),
	Tier_Second VARCHAR(40) NOT NULL,
	Battle_number INT NOT NULL,
    Did_the_first_pokemon_win INT NOT NULL
   	);''')
    

In [77]:
poke_df = pd.read_csv("Resources/final_df.csv")

In [78]:
poke_df.head()


,_ID,Number_First,First_Name,Type_1_First,Type_2_First,HP_First,Attack_First,Defense_First,Sp_Atk_First,Sp_Def_First,...,Attack_Second,Defense_Second,Sp_Atk_Second,Sp_Def_Second,Speed_Second,Generation_Second,Legendary_Second,Tier_Second,Battle_number,Did_the_first_pokemon_win
0,65,372,Barboach,Water,Ground,50,48,43,46,41,...,49,49,65,65,45,1,False,LC,1,1
1,42104,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,48,43,46,41,60,3,False,LC,1,0
2,79,462,Mothim,Bug,Flying,70,94,50,94,50,...,49,49,65,65,45,1,False,LC,2,1
3,51724,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,94,50,94,50,66,4,False,Untiered,2,0
4,0,6,Charmeleon,Fire,NaN,58,64,58,80,65,...,49,49,65,65,45,1,False,LC,3,1


In [79]:
poke_df.to_sql("pokemon", conn, if_exists='append', index = False)

In [80]:
c.execute('''SELECT * from pokemon''').fetchone()

(65,
 372,
 'Barboach',
 'Water',
 'Ground',
 50,
 48,
 43,
 46,
 41,
 60,
 3,
 '0',
 'LC',
 1,
 'Bulbasaur',
 'Grass',
 'Poison',
 45,
 49,
 49,
 65,
 65,
 45,
 1,
 '0',
 'LC',
 1,
 1)

In [81]:
conn.close()